In [2]:
import warnings
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import matplotlib
import time
from IPython.display import Image
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import seaborn as sns

In [3]:
df = pd.read_csv("C:/Users/Ngan/Desktop/dataset1_NaMi - FINAL.csv", encoding='cp1252')

In [4]:
df.head()

,Client_Num,Client_Gender,Client_Marital_Status_Grp,Client_Region_Tier,Client_Region_Longitude,Product_type
0,309,1,1,1.000000,105.834160,ILP
1,1743,0,1,1.000000,105.834160,ILP
2,2013,0,1,106.700883,10.776474,ILP
3,2158,1,1,1.000000,106.700883,Endownment
4,2496,0,1,1.000000,106.700883,PruCare


In [20]:
df.drop('Client_Region_Longitude', axis=1, inplace=True)


In [21]:
df.head()

,Client_Num,Product_type
0,309,ILP
1,1743,ILP
2,2013,ILP
3,2158,Endownment
4,2496,PruCare


In [22]:
df['Product_type'] = df['Product_type'].str.strip()

In [37]:
basket = df.groupby('Client_Num')['Product_type'].apply(list)

In [38]:
print(basket.head())

Client_Num
309            [ILP]
1743           [ILP]
2013           [ILP]
2158    [Endownment]
2496       [PruCare]
Name: Product_type, dtype: object


In [24]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [46]:
basket.values

array([list(['ILP']), list(['ILP']), list(['ILP']), ...,
       list(['Endownment', 'ILP']), list(['Endownment']),
       list(['Endownment'])], dtype=object)

In [47]:
dataset=basket.values
print(dataset)

[list(['ILP']) list(['ILP']) list(['ILP']) ... list(['Endownment', 'ILP'])
 list(['Endownment']) list(['Endownment'])]


In [58]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)

frequent_itemsets


,support,itemsets
0,0.412231,(Endownment)
1,0.513785,(ILP)
2,0.043944,(PruCare)
3,0.092176,(Universal)
4,0.033048,"(Endownment, ILP)"
5,0.006490,"(PruCare, Endownment)"
6,0.005389,"(Endownment, Universal)"
7,0.011730,"(PruCare, ILP)"
8,0.008009,"(Universal, ILP)"
9,0.001637,"(PruCare, Universal)"


In [59]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Endownment),(ILP),0.412231,0.513785,0.033048,0.080168,0.156033,-0.178750,0.528591
1,(ILP),(Endownment),0.513785,0.412231,0.033048,0.064322,0.156033,-0.178750,0.628175
2,(PruCare),(Endownment),0.043944,0.412231,0.006490,0.147696,0.358286,-0.011625,0.689625
3,(Universal),(Endownment),0.092176,0.412231,0.005389,0.058463,0.141820,-0.032609,0.624266
4,(PruCare),(ILP),0.043944,0.513785,0.011730,0.266938,0.519552,-0.010848,0.663266
5,(Universal),(ILP),0.092176,0.513785,0.008009,0.086886,0.169110,-0.039350,0.532481
6,"(PruCare, Endownment)",(ILP),0.006490,0.513785,0.002292,0.353211,0.687469,-0.001042,0.751737
7,"(PruCare, ILP)",(Endownment),0.011730,0.412231,0.002292,0.195431,0.474083,-0.002543,0.730540
8,"(Endownment, ILP)",(PruCare),0.033048,0.043944,0.002292,0.069369,1.578576,0.000840,1.027320
9,(PruCare),"(Endownment, ILP)",0.043944,0.033048,0.002292,0.052168,1.578576,0.000840,1.020173


In [40]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules[1:50]

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set